In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
df = pd.read_csv(r"C:\Users\mayan\Downloads\train.csv\train.csv")
test = pd.read_csv(r"C:\Users\mayan\Downloads\fake-news\test.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.fillna('')
test=test.fillna('')

In [5]:
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']

In [6]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [7]:
voc_size = 5000
msg = X.copy()
msg_test = test.copy()

In [8]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [10]:
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',msg['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [11]:
corpus_test = []
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [12]:
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [13]:
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [14]:
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 25, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 64)                6464      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [15]:
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (5200, 25))

In [16]:
model.fit(X_final,y_final,epochs=10,batch_size=64)

Epoch 1/10
325/325 [==============================] - 9s 17ms/step - loss: 0.1592 - accuracy: 0.9350
Epoch 2/10
325/325 [==============================] - 6s 19ms/step - loss: 0.0197 - accuracy: 0.9942
Epoch 3/10
325/325 [==============================] - 8s 24ms/step - loss: 0.0102 - accuracy: 0.9973
Epoch 4/10
325/325 [==============================] - 7s 23ms/step - loss: 0.0054 - accuracy: 0.9984
Epoch 5/10
325/325 [==============================] - 8s 24ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 6/10
325/325 [==============================] - 8s 23ms/step - loss: 0.0028 - accuracy: 0.9989
Epoch 7/10
325/325 [==============================] - 7s 21ms/step - loss: 0.0027 - accuracy: 0.9991
Epoch 8/10
325/325 [==============================] - 7s 22ms/step - loss: 0.0026 - accuracy: 0.9992
Epoch 9/10
325/325 [==============================] - 6s 18ms/step - loss: 0.0013 - accuracy: 0.9996
Epoch 10/10
325/325 [==============================] - 6s 17ms/step - loss: 0.0016 - accura

In [17]:
y_pred = np.argmax(model.predict(test_final), axis=-1)

163/163 [==============================] - 1s 4ms/step


In [23]:
final_sub = pd.DataFrame()
final_sub['id']=test['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)

In [24]:
final_sub.head()

,id,label
0,20800,0
1,20801,0
2,20802,0
3,20803,0
4,20804,0


In [27]:
model.fit(test_final,y_pred,epochs=10,batch_size=64)

Epoch 1/10
82/82 [==============================] - 3s 20ms/step - loss: 0.2635 - accuracy: 0.9615
Epoch 2/10
82/82 [==============================] - 2s 23ms/step - loss: 1.5626e-08 - accuracy: 1.0000
Epoch 3/10
82/82 [==============================] - 2s 22ms/step - loss: 6.8277e-10 - accuracy: 1.0000
Epoch 4/10
82/82 [==============================] - 2s 23ms/step - loss: 7.2569e-10 - accuracy: 1.0000
Epoch 5/10
82/82 [==============================] - 2s 22ms/step - loss: 1.0598e-09 - accuracy: 1.0000
Epoch 6/10
82/82 [==============================] - 2s 24ms/step - loss: 1.1446e-09 - accuracy: 1.0000
Epoch 7/10
82/82 [==============================] - 2s 22ms/step - loss: 4.2291e-10 - accuracy: 1.0000
Epoch 8/10
82/82 [==============================] - 2s 22ms/step - loss: 1.3507e-09 - accuracy: 1.0000
Epoch 9/10
82/82 [==============================] - 2s 26ms/step - loss: 5.8985e-09 - accuracy: 1.0000
Epoch 10/10
82/82 [==============================] - 2s 27ms/step - loss: 1.8